Steps
- Load settings from file
- Install / update Microsoft Graph PowerShell SDK (if needed)
- Authenticate to Microsoft Graph
- Create connection
- Register schema
- Ingest items
- Add activities
- Update permissions (if needed)

In [1]:
$settings = get-content .\appsettings.json | ConvertFrom-Json

$tenantId = $settings.tenantId
$clientId = $settings.clientId
$connectionId = $settings.connectionId
$connectionName = $settings.connectionName
$connectionDescription = $settings.connectionDescription
$certname = $settings.certificateName


In [2]:
$cert = Get-ChildItem -Path "Cert:\CurrentUser\My" | Where-Object {$_.Subject -Match "$certname"}

if(!$cert)
{
    # generate a self-signed certificate 
    $cert = New-SelfSignedCertificate -Subject "CN=$certname" -CertStoreLocation "Cert:\CurrentUser\My" -KeyExportPolicy Exportable -KeySpec Signature -KeyLength 2048 -KeyAlgorithm RSA -HashAlgorithm SHA256
    Export-Certificate -Cert $cert -FilePath "C:\scratch\$certname.cer"   ## Specify your preferred location

    # manually upload certificate to Entra ID application as a credential
}

In [ ]:
Export-Certificate -Cert $cert -FilePath "C:\scratch\$certname.cer"   ## Specify your preferred location

Install Microsoft Graph PowerShell SDK (if needed)

In [ ]:
#Install-Module -Name Microsoft.Graph
#Install-Module -Name Microsoft.Graph.Beta

#Update-Module -Name Microsoft.Graph
#Update-Module -Name Microsoft.Graph.Beta

Authenticate to Microsoft Graph

In [3]:
Connect-MgGraph -TenantId $tenantId -ClientId $clientId -CertificateThumbprint $($cert.Thumbprint) -NoWelcome

Create connection

In [ ]:
New-MgExternalConnection -Id $connectionId -Name $connectionName -Description $connectionDescription

Register schema

In [ ]:
$properties = @"
{
    "baseType": "microsoft.graph.externalItem",
    "properties": [
        {
            "name": "id",
            "type": "string",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": false,
            "isRefinable": false,
            "labels": [],
            "aliases": []
        },
        {
            "name": "title",
            "type": "string",
            "isSearchable": true,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": false,
            "labels": [
                "title"
            ],
            "aliases": []
        },
        {
            "name": "extension",
            "type": "string",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": true,
            "labels": [],
            "aliases": []
        },
        {
            "name": "createdBy",
            "type": "string",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": true,
            "labels": [
                "createdBy"
            ],
            "aliases": []
        },
        {
            "name": "createdDateTime",
            "type": "dateTime",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": false,
            "labels": [
                "createdDateTime"
            ],
            "aliases": []
        },
        {
            "name": "lastModifiedBy",
            "type": "string",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": false,
            "labels": [
                "lastModifiedBy"
            ],
            "aliases": []
        },
        {
            "name": "lastModifiedDate",
            "type": "dateTime",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": true,
            "labels": [
                "lastModifiedDateTime"
            ],
            "aliases": []
        },
        {
            "name": "url",
            "type": "string",
            "isSearchable": false,
            "isRetrievable": true,
            "isQueryable": false,
            "isRefinable": false,
            "labels": [
                "url"
            ],
            "aliases": []
        },
        {
            "name": "description",
            "type": "string",
            "isSearchable": true,
            "isRetrievable": true,
            "isQueryable": false,
            "isRefinable": false,
            "labels": [],
            "aliases": []
        },
        {
            "name": "authors",
            "type": "stringCollection",
            "isSearchable": true,
            "isRetrievable": true,
            "isQueryable": true,
            "isRefinable": false,
            "labels": [
                "authors"
            ],
            "aliases": []
        },
        {
            "type": "String",
            "name": "iconUrl",
            "isSearchable": false,
            "isQueryable": false,
            "isRetrievable": true,
            "isRefinable": false,
            "labels": [
                "iconUrl"
            ],
            "aliases": []
        },
        {
            "type": "String",
            "name": "imageUrl",
            "isSearchable": false,
            "isQueryable": false,
            "isRetrievable": true,
            "isRefinable": false,
            "labels": [],
            "aliases": []
        },
        {
            "type": "String",
            "name": "containerName",
            "isSearchable": false,
            "isQueryable": false,
            "isRetrievable": true,
            "isRefinable": false,
            "labels": [
                "containerName"
            ],
            "aliases": []
        },
        {
            "type": "String",
            "name": "containerUrl",
            "isSearchable": false,
            "isQueryable": false,
            "isRetrievable": true,
            "isRefinable": false,
            "labels": [
                "containerUrl"
            ],
            "aliases": []
        }
    ]
}
"@

# issues running this command so falling back to manual Graph request following
#$propertiesBody = $properties | ConvertFrom-Json
#Update-MgExternalConnectionSchema -ExternalConnectionId $connectionId -properties $propertiesBody.properties -BaseType $propertiesBody.baseType

Invoke-MgGraphRequest -Method PATCH -Uri "/v1.0/external/connections/$connectionId/schema" -Body $properties


Ingest items

Add activities

Update permissions (ACL = access control list)